In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\AdrienSourdille\Desktop\Anglure-Windfarm-Impact\data\processed\dvf_vente_51_distances.csv")

display(df.head())

C:\Users\AdrienSourdille\AppData\Local\Temp\ipykernel_22192\3846857583.py:3: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\AdrienSourdille\Desktop\Anglure-Windfarm-Impact\data\processed\dvf_vente_51_distances.csv")


,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,price_per_m2,distance_km,distance_bucket
0,2014-536311,2014-01-14,2,Vente,125500.0,72.0,NaN,RUE DES ROSES,0193,51130.0,...,S,sols,NaN,NaN,297.0,4.002311,48.899476,1410.112360,32.660350,30-50 km
1,2014-536313,2014-01-15,1,Vente,200000.0,1.0,NaN,RUE DU SAULE BLANC,0063,51130.0,...,S,sols,NaN,NaN,2836.0,3.991857,48.812734,1652.892562,24.188717,20-30 km
2,2014-536381,2014-02-04,1,Vente,38200.0,25.0,NaN,BD CARNOT,0016,51130.0,...,S,sols,NaN,NaN,304.0,4.002688,48.903391,217.045455,33.054949,30-50 km
3,2014-536395,2014-02-14,1,Vente,133000.0,2.0,NaN,RUE DE L EGLISE,0010,51320.0,...,S,sols,NaN,NaN,1100.0,4.401544,48.779923,1176.991150,47.724887,30-50 km
4,2014-536413,2014-02-14,1,Vente,156320.0,33.0,NaN,RUE NATIONALE,0090,51130.0,...,S,sols,NaN,NaN,965.0,4.004281,48.877517,2004.102564,30.623527,30-50 km


In [ ]:
import statsmodels.api as sm

# Drop rows with missing values
df_model = df[["valeur_fonciere", "surface_reelle_bati", "distance_km"]].dropna()

# Define features and target
X = df_model[["surface_reelle_bati", "distance_km"]]
X = sm.add_constant(X)  # Adds intercept
y = df_model["valeur_fonciere"]

# Fit the model
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:        valeur_fonciere   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.178
Date:                Sun, 08 Jun 2025   Prob (F-statistic):            0.00208
Time:                        21:00:29   Log-Likelihood:            -2.8116e+05
No. Observations:               16941   AIC:                         5.623e+05
Df Residuals:                   16938   BIC:                         5.624e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -4.326e+04   1

In [20]:
import numpy as np
import pandas as pd

# Define post-treatment indicator (when construction begins)
df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
df['post'] = (df['date_mutation'] >= "2018-10-01").astype(int)

# Define treated group: within 3 km of windfarm
df['treated'] = (df['distance_km'] <= 3).astype(int)

# Diff-in-diff interaction
df['did'] = df['post'] * df['treated']

# Log price
df['log_valeur_fonciere'] = np.log(df['valeur_fonciere'])

# Precompute year
df['year'] = df['date_mutation'].dt.year

# Drop rows with missing values in relevant columns before fitting
df_model = df.dropna(subset=[
    'log_valeur_fonciere', 'post', 'treated', 'did', 
    'surface_reelle_bati', 'surface_terrain', 'type_local', 
    'code_commune', 'year'
])



In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16977 entries, 0 to 16976
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id_mutation                   16977 non-null  object        
 1   date_mutation                 16977 non-null  datetime64[ns]
 2   numero_disposition            16977 non-null  int64         
 3   nature_mutation               16977 non-null  object        
 4   valeur_fonciere               16942 non-null  float64       
 5   adresse_numero                16935 non-null  float64       
 6   adresse_suffixe               791 non-null    object        
 7   adresse_nom_voie              16977 non-null  object        
 8   adresse_code_voie             16977 non-null  object        
 9   code_postal                   16977 non-null  float64       
 10  code_commune                  16977 non-null  int64         
 11  nom_commune                 

In [15]:
df_model = df[['log_valeur_fonciere', 'post', 'treated', 'did', 'surface_reelle_bati', 'surface_terrain', 'type_local', 'code_commune']].dropna()
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14505 entries, 0 to 16976
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   log_valeur_fonciere  14505 non-null  float64
 1   post                 14505 non-null  int64  
 2   treated              14505 non-null  int64  
 3   did                  14505 non-null  int64  
 4   surface_reelle_bati  14505 non-null  float64
 5   surface_terrain      14505 non-null  float64
 6   type_local           14505 non-null  object 
 7   code_commune         14505 non-null  int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 1019.9+ KB


In [28]:
import statsmodels.formula.api as smf
import os

# Create output folder if it doesn't exist
os.makedirs("output", exist_ok=True)

model = smf.ols(
    "log_valeur_fonciere ~ post + treated + did + "
    "surface_reelle_bati + surface_terrain + C(type_local) + "
    "C(code_commune) + C(year)",
    data=df
).fit(cov_type='cluster', cov_kwds={'groups': df_model['code_commune']})

with open("../output/model_summary.txt", "w", encoding="utf-8") as f:
    f.write(model.summary().as_text())

c:\Users\AdrienSourdille\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 233, but rank is 15
  warnings.warn('covariance of constraints does not have full '
